In [1]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from xgboost.callback import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


random_state = 42
np.random.seed(random_state)

In [28]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv')
ctr_17 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_17.csv') 
ctr_18 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_18.csv')
ctr_19 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_19.csv') 
ctr_20 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_20.csv')    
data = pd.concat(([ctr_15,ctr_16,ctr_17,ctr_18,ctr_19,ctr_20]))
#data = ctr_15
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print("Tamaño del conjunto ", data.shape)
print("Tamaño del conjunto ", ctr_test.shape)


Tamaño del conjunto  (7121371, 52)
Tamaño del conjunto  (1139639, 51)


In [29]:
# Lista de columnas a eliminar
columns_to_remove = [
    'action_categorical_0', 'action_categorical_2', 'action_categorical_4', 
    'auction_categorical_0', 'auction_time', 
    'auction_categorical_10', 'action_categorical_7', 'action_categorical_6',
    'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10',
    'auction_categorical_4', 'gender', 'creative_categorical_1', 
    'auction_categorical_6', 'creative_categorical_3',
    'auction_categorical_2', 'action_categorical_5',
    'auction_categorical_1', 'auction_categorical_5', 'device_id_type',
    'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0',
    'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5',
    'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8',
    'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1',
    'creative_categorical_2', 'action_categorical_3', 'action_categorical_2',
    'auction_categorical_11', 'action_categorical_4',
    'creative_categorical_0', 'device_id', 'auction_boolean_0',
      #'action_list_2','action_list_1','action_list_0', 'auction_list_0'
]

ctr_test['creative_height_square'] = ctr_test['creative_height'] * ctr_test['creative_height']
ctr_test['creative_width_square'] = ctr_test['creative_width'] * ctr_test['creative_width']
ctr_test['creative_area'] = ctr_test['creative_height'] * ctr_test['creative_width']

data['creative_height_square'] = data['creative_height'] * data['creative_height']
data['creative_width_square'] = data['creative_width'] * data['creative_width']
data['creative_area'] = data['creative_height'] * data['creative_width']

data = data.drop(columns=columns_to_remove, errors='ignore')
ctr_test = ctr_test.drop(columns=columns_to_remove, errors='ignore')

print(f"Columnas eliminadas: {columns_to_remove}")
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")
data.info()



Columnas eliminadas: ['action_categorical_0', 'action_categorical_2', 'action_categorical_4', 'auction_categorical_0', 'auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6', 'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10', 'auction_categorical_4', 'gender', 'creative_categorical_1', 'auction_categorical_6', 'creative_categorical_3', 'auction_categorical_2', 'action_categorical_5', 'auction_categorical_1', 'auction_categorical_5', 'device_id_type', 'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0', 'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5', 'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8', 'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1', 'creative_categorical_2', 'action_categorical_3', 'action_categorical_2', 'auction_categorical_11', 'action_categorical_4', 'creative_categorical_0', 'device_id', 'auction_boolean_0']
T

In [30]:
def impute_count(value):
    if isinstance(value, list):  # Verificar si el valor es una lista
        return len(value)  # Devolver el número de elementos de la lista
    else:
        return 0  # Valor predeterminado si no es una lista

# Aplicar la función a las columnas correspondientes
data['action_list_2'] = data['action_list_2'].apply(impute_count)
ctr_test['action_list_2'] = ctr_test['action_list_2'].apply(impute_count)

data['action_list_1'] = data['action_list_1'].apply(impute_count)
ctr_test['action_list_1'] = ctr_test['action_list_1'].apply(impute_count)

data['action_list_0'] = data['action_list_0'].apply(impute_count)
ctr_test['action_list_0'] = ctr_test['action_list_0'].apply(impute_count)

data['auction_list_0'] = data['auction_list_0'].apply(impute_count)
ctr_test['auction_list_0'] = ctr_test['auction_list_0'].apply(impute_count)

# Mostrar información de los DataFrames
data.info()
ctr_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7121371 entries, 0 to 1356065
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Label                   int64  
 1   action_list_0           int64  
 2   action_list_1           int64  
 3   action_list_2           int64  
 4   auction_age             float64
 5   auction_bidfloor        float64
 6   auction_categorical_12  object 
 7   auction_categorical_3   object 
 8   auction_categorical_7   object 
 9   auction_categorical_9   object 
 10  auction_list_0          int64  
 11  creative_height         float64
 12  creative_width          float64
 13  has_video               bool   
 14  timezone_offset         float64
 15  creative_height_square  float64
 16  creative_width_square   float64
 17  creative_area           float64
dtypes: bool(1), float64(8), int64(5), object(4)
memory usage: 984.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139639 entries, 0 to 1139638

In [6]:
def impute_average(value):
    if isinstance(value, list):  # Verificar si el valor es una lista
        # Convertir todos los elementos de la lista a enteros, ignorando cualquier dato no convertible
        int_list = [int(x) for x in value if isinstance(x, (int, str)) and str(x).isdigit()]
        if len(int_list) == 0:
            return 0  # o cualquier valor predeterminado para listas vacías
        return int(sum(int_list) / len(int_list))
    else:
        return 0  # Valor predeterminado si no es una lista

# Aplicar la función a la columna 'action_list_2'
data['action_list_2'] = data['action_list_2'].apply(impute_average)
ctr_test['action_list_2'] = ctr_test['action_list_2'].apply(impute_average)

data['action_list_1'] = data['action_list_1'].apply(impute_average)
ctr_test['action_list_1'] = ctr_test['action_list_1'].apply(impute_average)

data['action_list_0'] = data['action_list_0'].apply(impute_average)
ctr_test['action_list_0'] = ctr_test['action_list_0'].apply(impute_average)

data['auction_list_0'] = data['auction_list_0'].apply(impute_average)
ctr_test['auction_list_0'] = ctr_test['auction_list_0'].apply(impute_average)

data.info()
ctr_test.info()


KeyError: 'action_list_2'

In [31]:
y = data[['Label']].copy() 
X = data.drop('Label', axis = 1)

X_train, X_val, Y_train, Y_val = train_test_split(X, y,
                                                  train_size = 0.8,
                                                  random_state = 32,
                                                  stratify = y)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")


Cantidad de datos de train: 5697096
Cantidad de datos de validación: 1424275
Tamaño del data después de la eliminación: (7121371, 18)
Tamaño del test después de la eliminación: (1139639, 17)


In [5]:
def fill_nans(df):
    numeric_columns = df.select_dtypes(include=['number']).columns
    for col in numeric_columns:
        mode_value = df[col].mode()[0] 
        df[col].fillna(mode_value, inplace=True)

    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        mode_value = df[col].mode()[0]
        df[col].fillna(mode_value, inplace=True)
    
    return df

X_train = fill_nans(X_train)
X_val = fill_nans(X_val)
ctr_test = fill_nans(ctr_test)

print(f"Tamaño del data después de rellenar NaNs: {data.shape}")
print(f"Tamaño del test después de rellenar NaNs: {X_val.shape}")
print(f"Tamaño del test después de rellenar NaNs: {ctr_test.shape}")

/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipykernel_54855/3841489051.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)
/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipykernel_54855/3841489051.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

Tamaño del data después de rellenar NaNs: (1239524, 10)
Tamaño del test después de rellenar NaNs: (247905, 9)
Tamaño del test después de rellenar NaNs: (1139639, 9)


/var/folders/m6/yrlm4ckj0r71yq2s7_1v54kh0000gn/T/ipykernel_54855/3841489051.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_value, inplace=True)


In [32]:
def group_rare_categories(column, threshold):
    freq = column.value_counts(normalize=True)
    rare_categories = freq[freq < threshold].index
    return column.apply(lambda x: 'Other' if x in rare_categories else x)

threshold = 0.05

categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()

if 'action_list_2' in categorical_columns:
    categorical_columns.remove('action_list_2')

for col in categorical_columns:
    X_train[col] = group_rare_categories(X_train[col], threshold)

pd_ohe = pd.get_dummies(X_train,
                        columns=categorical_columns + ['has_video'],
                        sparse=False,
                        dummy_na=False,
                        dtype=int)

print("Shape de data después de OneHotEncoding:", pd_ohe.shape)
pd_ohe.info()

Shape de data después de OneHotEncoding: (5697096, 24)
<class 'pandas.core.frame.DataFrame'>
Index: 5697096 entries, 975913 to 83486
Data columns (total 24 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   action_list_0                    int64  
 1   action_list_1                    int64  
 2   action_list_2                    int64  
 3   auction_age                      float64
 4   auction_bidfloor                 float64
 5   auction_list_0                   int64  
 6   creative_height                  float64
 7   creative_width                   float64
 8   timezone_offset                  float64
 9   creative_height_square           float64
 10  creative_width_square            float64
 11  creative_area                    float64
 12  auction_categorical_12_9c3324c7  int64  
 13  auction_categorical_12_Other     int64  
 14  auction_categorical_12_b4e520dd  int64  
 15  auction_categorical_3_Other      int64  
 16  a

In [33]:
missing_columns_validation = pd_ohe.columns.difference(X_val.columns)
missing_columns_test = pd_ohe.columns.difference(ctr_test.columns)

for col in missing_columns_validation:
    X_val[col] = 0
print("done validation")

for col in missing_columns_test:
    ctr_test[col] = 0
print("done test")

label_column = 'label'
common_columns_val = X_val.columns.intersection(pd_ohe.columns)
common_columns_test = ctr_test.columns.intersection(pd_ohe.columns.drop(label_column, errors='ignore'))

X_val = X_val[common_columns_val]
ctr_test = ctr_test[common_columns_test]

if label_column in ctr_test.columns:
    ctr_test = ctr_test.drop(columns=[label_column])

print(pd_ohe.shape)
print(X_val.shape)
print(ctr_test.shape)

print(pd_ohe.dtypes)
print(ctr_test.dtypes)


done validation
done test
(5697096, 24)
(1424275, 24)
(1139639, 24)
action_list_0                        int64
action_list_1                        int64
action_list_2                        int64
auction_age                        float64
auction_bidfloor                   float64
auction_list_0                       int64
creative_height                    float64
creative_width                     float64
timezone_offset                    float64
creative_height_square             float64
creative_width_square              float64
creative_area                      float64
auction_categorical_12_9c3324c7      int64
auction_categorical_12_Other         int64
auction_categorical_12_b4e520dd      int64
auction_categorical_3_Other          int64
auction_categorical_7_034e01d7       int64
auction_categorical_7_Other          int64
auction_categorical_7_b04e4d07       int64
auction_categorical_9_11105871       int64
auction_categorical_9_Other          int64
auction_categorical_9_d4eea9a

In [18]:
params = {'max_depth': list(range(1, 40)),
          'learning_rate': uniform(scale = 0.2),
          'gamma': uniform(scale = 2),
          'reg_lambda': uniform(scale = 5),        # Parámetro de regularización.
          'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
          'min_child_weight': uniform(scale = 5),
          'colsample_bytree': uniform(0.75, 0.25), # Entre 0.75 y 1.
          'n_estimators': list(range(1, 1000))
         }

start = time.time()
best_score = 0
best_estimator = None
iterations = 10
current_iteration = 1  # Inicia el contador de iteraciones

for g in ParameterSampler(params, n_iter=iterations, random_state=32):
    print(f'Iteración {current_iteration} de {iterations}')  # Imprime la iteración actual

    clf_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=32, eval_metric='auc', tree_method='hist', device='cuda', **g)
    clf_xgb.fit(pd_ohe, Y_train, eval_set=[(X_val, Y_val)], verbose=False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1]  # Obtenemos la probabilidad de una de las clases (cualquiera).
    auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)

    # Guardamos si es mejor.
    if auc_roc > best_score:
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

    current_iteration += 1  # Aumenta el contador de iteraciones

end = time.time()
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo transcurrido: {str(end - start)} segundos')
print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

Iteración 1 de 10


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x158854e30>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py", line 582, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 


XGBoostError: [01:51:46] /Users/runner/work/xgboost/xgboost/src/common/quantile.h:770: Check failed: count <= total_entries (3551300 vs. 0) : 
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000146ff858c dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000147104744 std::__1::vector<unsigned int, std::__1::allocator<unsigned int>> xgboost::common::LoadBalance<xgboost::data::ColumnarAdapterBatch, xgboost::data::IsValidFunctor&>(xgboost::data::ColumnarAdapterBatch const&, unsigned long, unsigned int, unsigned long, xgboost::data::IsValidFunctor&) + 432
  [bt] (2) 3   libxgboost.dylib                    0x0000000147104474 void xgboost::common::SketchContainerImpl<xgboost::common::WQuantileSketch<float, float>>::PushRowPageImpl<xgboost::data::ColumnarAdapterBatch, xgboost::data::IsValidFunctor>(xgboost::data::ColumnarAdapterBatch const&, unsigned long, xgboost::common::OptionalWeights, unsigned long, unsigned long

In [40]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',
                            use_label_encoder=False,
                            colsample_bytree=0.7732274867094971, 
                            gamma= 1.8050427802244318, 
                            learning_rate= 0.09219208117648266, 
                            max_depth= 7, 
                            min_child_weight= 0.006491690852697407, 
                            n_estimators=100, 
                            reg_lambda= 0.812118717763774, 
                            subsample= 0.8546852920995762) 

clf_xgb.fit(pd_ohe, Y_train, verbose=True, eval_set=[(X_val, Y_val)])

best_auc = 0
best_iteration = 0
y_pred_best = None

/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [02:06:22] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.80093
[1]	validation_0-auc:0.80744
[2]	validation_0-auc:0.80864
[3]	validation_0-auc:0.80659
[4]	validation_0-auc:0.81013
[5]	validation_0-auc:0.81168
[6]	validation_0-auc:0.81108
[7]	validation_0-auc:0.81126
[8]	validation_0-auc:0.81443
[9]	validation_0-auc:0.81347
[10]	validation_0-auc:0.81606
[11]	validation_0-auc:0.81609
[12]	validation_0-auc:0.81560
[13]	validation_0-auc:0.81456
[14]	validation_0-auc:0.81281
[15]	validation_0-auc:0.81343
[16]	validation_0-auc:0.81492
[17]	validation_0-auc:0.81491
[18]	validation_0-auc:0.81485
[19]	validation_0-auc:0.81550
[20]	validation_0-auc:0.81531
[21]	validation_0-auc:0.81461
[22]	validation_0-auc:0.81392
[23]	validation_0-auc:0.81268
[24]	validation_0-auc:0.81365
[25]	validation_0-auc:0.81282
[26]	validation_0-auc:0.81367
[27]	validation_0-auc:0.81454
[28]	validation_0-auc:0.81453
[29]	validation_0-auc:0.81375
[30]	validation_0-auc:0.81359
[31]	validation_0-auc:0.81326
[32]	validation_0-auc:0.81426
[33]	validation_0-au

In [41]:
importances = clf_xgb.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': pd_ohe.columns, 'Importance': importances})
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)

threshold = 0.001  # Por ejemplo, eliminar características con importancia menor al 1%
features_to_keep = feature_importance_df[feature_importance_df['Importance'] >= threshold]['Feature']
pd_ohe = pd_ohe[features_to_keep]
X_val = X_val[features_to_keep]
ctr_test = ctr_test[features_to_keep]
pd_ohe.shape

(5697096, 12)

In [42]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',
                            use_label_encoder=False,
                            colsample_bytree=0.7732274867094971, 
                            gamma= 1.8050427802244318, 
                            learning_rate= 0.09219208117648266, 
                            max_depth= 7, 
                            min_child_weight= 0.006491690852697407, 
                            n_estimators=300, 
                            reg_lambda= 0.812118717763774, 
                            subsample= 0.8546852920995762) 

clf_xgb.fit(pd_ohe, Y_train, verbose=True, eval_set=[(X_val, Y_val)])

best_auc = 0
best_iteration = 0
y_pred_best = None

/opt/anaconda3/envs/TD6/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [02:06:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.78774
[1]	validation_0-auc:0.78713
[2]	validation_0-auc:0.81373
[3]	validation_0-auc:0.81095
[4]	validation_0-auc:0.81364
[5]	validation_0-auc:0.81499
[6]	validation_0-auc:0.81415
[7]	validation_0-auc:0.81469
[8]	validation_0-auc:0.81405
[9]	validation_0-auc:0.81387
[10]	validation_0-auc:0.81220
[11]	validation_0-auc:0.81222
[12]	validation_0-auc:0.81267
[13]	validation_0-auc:0.81214
[14]	validation_0-auc:0.81208
[15]	validation_0-auc:0.81247
[16]	validation_0-auc:0.81282
[17]	validation_0-auc:0.81407
[18]	validation_0-auc:0.81467
[19]	validation_0-auc:0.81444
[20]	validation_0-auc:0.81515
[21]	validation_0-auc:0.81479
[22]	validation_0-auc:0.81610
[23]	validation_0-auc:0.81507
[24]	validation_0-auc:0.81574
[25]	validation_0-auc:0.81523
[26]	validation_0-auc:0.81532
[27]	validation_0-auc:0.81489
[28]	validation_0-auc:0.81520
[29]	validation_0-auc:0.81540
[30]	validation_0-auc:0.81464
[31]	validation_0-auc:0.81414
[32]	validation_0-auc:0.81286
[33]	validation_0-au

In [43]:
ctr_test['id'] = range(1, len(ctr_test) + 1)
y_pred_test = clf_xgb.predict_proba(ctr_test.drop(columns=["id"]))[:, clf_xgb.classes_ == 1].squeeze()
submission_df = pd.DataFrame({"id": ctr_test["id"], "Label": y_pred_test})
submission_df["id"] = submission_df["id"].astype(int)
submission_df.to_csv("test_xgboost.csv", sep=",", index=False)